In [1]:
Kaggle = False
Colab = True

In [2]:
import os, sys
from pathlib import Path

if Colab:
    from google.colab import drive
    drive.mount('/content/drive')

    path = "/content/drive/My Drive"

    os.chdir(path)
    os.listdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#  if Kaggle:
#     !rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
#     !git clone --recursive https://github.com/Microsoft/LightGBM
#     !apt-get install -y -qq libboost-all-dev
#     %%bash
#     cd LightGBM
#     rm -r build
#     mkdir build
#     cd build
#     cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
#     make -j$(nproc)
#     !cd LightGBM/python-package/;python3 setup.py install --precompile
#     !mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
#     !rm -r LightGBM
# else:

# !git clone — recursive https://github.com/Microsoft/LightGBM
# %cd /content/LightGBM
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install — precompile

In [4]:
# !git clone https://github.com/Microsoft/LightGBM
# %cd /content/drive/My\ Drive/LightGBM/
# !mkdir build
# !cmake -DUSE_GPU=1 #avoid ..
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U

In [5]:
%cd /content/drive/My\ Drive/LightGBM/python-package
!sudo python setup.py install

/content/drive/My Drive/LightGBM/python-package
running install
INFO:LightGBM:Starting to compile the library.
INFO:LightGBM:Starting to compile with CMake.
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
running install_lib
INFO:LightGBM:Installing lib_lightgbm from: ['../lib_lightgbm.so', 'compile/lib_lightgbm.so']
running install_egg_info
removing '/usr/local/lib/python3.6/dist-packages/lightgbm-2.3.2-py3.6.egg-info' (and everything under it)
Copying lightgb

In [6]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import lightgbm as lgb

> This notebook aims to push the public LB under 0.50. Certainly, the competition is not yet at its peak and there clearly remains room for improvement.

# Credits

* [First R notebook](https://www.kaggle.com/kailex/m5-forecaster-v2)
* [Python translation](https://www.kaggle.com/kneroma/m5-forecast-v2-python)

# Changes
* v5 : try to optimise the LGBM params (go below in lgbm params section to see changes)
* v4 : add df, X_train deletion before training step --> increasing train sample without memeroy issues

In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Fri Jun 19 21:45:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
%cd /content/drive/My\ Drive/

/content/drive/My Drive


In [9]:
if Kaggle:
    PATH = '/kaggle/input/'
    outdir = '.'
# PATH = '/Users/helen/Desktop/Data/'
else:
    PATH = 'm5_competition/'
    outdir = Path(PATH+'res')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outdir = Path(PATH+'res/lgbm')
    if not os.path.exists(outdir):
        os.mkdir(outdir)

In [10]:
MODELNAME = 'LGBM-GPU'
VERSION = '{}'.format(MODELNAME)
outdir = os.path.join(outdir, VERSION)
if not os.path.exists(outdir):
    os.mkdir(outdir)

from datetime import datetime
dateTimeObj = datetime.now()
# timestampStr = dateTimeObj.strftime("%d-%b-%Y-%H")
model_path = os.path.join(outdir, '19-Jun-2020-19-l28-w28')
timestampStr = '19-Jun-2020-19-l28-w28-2'
outdir = os.path.join(outdir, timestampStr)
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [11]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "int16", 'snap_TX': 'int16', 'snap_WI': 'int16' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [12]:
pd.options.display.max_columns = 50

In [13]:
h = 28 
max_lags = 57
tr_last = 1913
fday = datetime(2016,4, 25) 
fday

datetime.datetime(2016, 4, 25, 0, 0)

In [14]:

def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if (c_min > np.iinfo(np.int16).min
                      and c_max < np.iinfo(np.int16).max):
                    df[col] = df[col].astype(np.int16)
                elif (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df


In [15]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    prices = pd.read_csv(PATH+ "m5-forecasting-accuracy/sell_prices.csv", dtype = PRICE_DTYPES)
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
            
    cal = pd.read_csv(PATH+"m5-forecasting-accuracy/calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    display(cal.head())   
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    # dtype.update({col: "category" for col in catcols})
    dt = pd.read_csv(PATH+"m5-forecasting-accuracy/sales_train_validation.csv", 
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)
    
    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()

    # for col in catcols:
    #     dt[col] = dt[col].cat.codes.astype("int16")
    #     dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    # print("step 0")
    # display(dt.head())
    dt = dt.merge(cal, on= "d", copy = False)
    # print("step 1")
    # dt['d'] = dt['d'].astype('category').cat.codes.astype("int16")
    # print(dt['d'].min(), dt['d'].max())
    # print(np.unique(dt['d']))
    # print(len(np.unique(dt['d'])))
    # dt['d'] -= dt['d'].min()
    # print("step 1")
    # display(dt.head())
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)
    display(dt.head())
    return dt

In [16]:
def create_fea(dt):
    lags = [7, 28]
    lag_cols = [f"lag_{lag}" for lag in lags ]
    for lag, lag_col in zip(lags, lag_cols):
        dt[lag_col] = dt[["id","sales"]].groupby("id")["sales"].shift(lag)

    wins = [7, 28]
    for win in wins :
        for lag,lag_col in zip(lags, lag_cols):
            dt[f"rmean_{lag}_{win}"] = dt[["id", lag_col]].groupby("id")[lag_col].transform(lambda x : x.rolling(win).mean())

    
    
    date_features = {
        
        "wday": "weekday",
        "week": "weekofyear",
        "month": "month",
        "quarter": "quarter",
        "year": "year",
        "mday": "day",
#         "ime": "is_month_end",
#         "ims": "is_month_start",
    }
    
#     dt.drop(["d", "wm_yr_wk", "weekday"], axis=1, inplace = True)
    
    for date_feat_name, date_feat_func in date_features.items():
        if date_feat_name in dt.columns:
            dt[date_feat_name] = dt[date_feat_name].astype("int16")
        else:
            dt[date_feat_name] = getattr(dt["date"].dt, date_feat_func).astype("int16")

In [17]:
FIRST_DAY = 350 # If you want to load all the data set it to '1' -->  Great  memory overflow  risk !

In [18]:
%%time

df = create_dt(is_train=True, first_day= FIRST_DAY)
df = reduce_mem_usage(df)
df.shape

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.97
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.34
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.48
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.50
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.77


Mem. usage decreased to 2718.23 MB (5.4 % reduction)
CPU times: user 33.1 s, sys: 2.61 s, total: 35.7 s
Wall time: 35.9 s


In [19]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.970703
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.339844
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.480469
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.500000
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.769531


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 22 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float16       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       int16         
 19  snap_TX       int16         
 20  snap_WI       int16         
 21  sell_price    float16       
dtypes: datetime64[ns](1), float16(2), int16(17), object(2)
memory us

In [21]:
%%time

create_fea(df)
df.shape
df = reduce_mem_usage(df)

Mem. usage decreased to 3417.21 MB (0.0 % reduction)
CPU times: user 3min 17s, sys: 884 ms, total: 3min 18s
Wall time: 3min 18s


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40718219 entries, 0 to 40718218
Data columns (total 31 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   d             object        
 7   sales         float16       
 8   date          datetime64[ns]
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  wday          int16         
 12  month         int16         
 13  year          int16         
 14  event_name_1  int16         
 15  event_type_1  int16         
 16  event_name_2  int16         
 17  event_type_2  int16         
 18  snap_CA       int16         
 19  snap_TX       int16         
 20  snap_WI       int16         
 21  sell_price    float16       
 22  lag_7         float16       
 23  lag_28        float16       
 

In [23]:
df.head()

,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
0,HOBBIES_1_002_CA_1_validation,1,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
1,HOBBIES_1_004_CA_1_validation,3,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,4.339844,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
2,HOBBIES_1_005_CA_1_validation,4,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,2.480469,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
3,HOBBIES_1_008_CA_1_validation,7,0,0,0,0,d_350,0.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,0.500000,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13
4,HOBBIES_1_009_CA_1_validation,8,0,0,0,0,d_350,2.0,2012-01-13,11150,0,7,1,2012,0,0,0,0,0,1,0,1.769531,NaN,NaN,NaN,NaN,NaN,NaN,2,1,13


In [24]:
df.dropna(inplace = True)
df.shape

(39041269, 31)

In [25]:
cat_feats = ['item_id', 'dept_id','store_id', 'cat_id', 'state_id'] + ["event_name_1", "event_name_2", "event_type_1", "event_type_2"]
useless_cols = ["id", "date", "sales","d", "wm_yr_wk", "weekday"]
train_cols = df.columns[~df.columns.isin(useless_cols)]
X_train = df[train_cols]
y_train = df["sales"]

In [26]:
display(X_train.head())


,item_id,dept_id,store_id,cat_id,state_id,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,lag_7,lag_28,rmean_7_7,rmean_28_7,rmean_7_28,rmean_28_28,week,quarter,mday
869062,1,0,0,0,0,6,3,2012,24,3,0,0,1,0,1,3.970703,0.0,0.0,0.000000,0.142822,0.214233,0.214233,10,1,8
869063,1,0,0,0,0,7,3,2012,0,0,0,0,1,1,1,3.970703,0.0,1.0,0.000000,0.142822,0.178589,0.250000,10,1,9
869069,3,0,0,0,0,6,3,2012,24,3,0,0,1,0,1,4.339844,3.0,2.0,1.857422,1.142578,1.392578,1.678711,10,1,8
869070,3,0,0,0,0,7,3,2012,0,0,0,0,1,1,1,4.339844,1.0,0.0,2.000000,1.000000,1.392578,1.607422,10,1,9
869076,4,0,0,0,0,6,3,2012,24,3,0,0,1,0,1,2.980469,0.0,0.0,0.000000,1.857422,0.750000,0.643066,10,1,8


In [27]:
display(y_train.head())

869062    0.0
869063    0.0
869069    1.0
869070    2.0
869076    0.0
Name: sales, dtype: float16

In [28]:
# train_data = lgb.Dataset(X_train, label = y_train, categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_inds = np.random.choice(len(X_train), 1000000, replace = False)
# fake_valid_data = lgb.Dataset(X_train.iloc[fake_valid_inds], label = y_train.iloc[fake_valid_inds],categorical_feature=cat_feats,
#                              free_raw_data=False)   # This is just a subsample of the training set, not a real validation set !

In [29]:
%%time

np.random.seed(777)

# fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
# train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
# train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds], 
#                          categorical_feature=cat_feats, free_raw_data=False)
# fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
#                               categorical_feature=cat_feats,
#                  free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

fake_valid_inds = np.random.choice(X_train.index.values, 2_000_000, replace = False)
train_inds = np.setdiff1d(X_train.index.values, fake_valid_inds)
train_data = lgb.Dataset(X_train.loc[train_inds] , label = y_train.loc[train_inds],free_raw_data=False)
fake_valid_data = lgb.Dataset(X_train.loc[fake_valid_inds], label = y_train.loc[fake_valid_inds],
                 free_raw_data=False)# This is a random sample, we're not gonna apply any time series train-test-split tricks here!

CPU times: user 17.4 s, sys: 540 ms, total: 17.9 s
Wall time: 17.9 s


In [30]:
del df, X_train, y_train, fake_valid_inds,train_inds ; gc.collect()

43

In [31]:
params = {
        "objective" : "poisson",
        "metric" :"rmse",
        "force_row_wise" : True,
        "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
        "sub_row" : 0.75,
        "bagging_freq" : 1,
        "lambda_l2" : 0.1,
        "nthread" : 4,
        "metric": ["rmse"],
    'verbosity': 1,
    'num_iterations' : 1200,
    'num_leaves': 256,
    "min_data_in_leaf": 100,
    'device': 'gpu'
}

In [32]:
%%time

# m_lgb = lgb.train(params, train_data, valid_sets = [fake_valid_data], verbose_eval=20) 

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs


In [33]:
# m_lgb.save_model(os.path.join(outdir,"model-l256-fd{}.lgb".format(FIRST_DAY)))

In [35]:
m_lgb = lgb.Booster(model_file=(os.path.join(model_path,"model-fd{}.lgb".format(FIRST_DAY))))
print("model loaded")

model loaded


In [36]:
%%time

alphas = [1.028, 1.023, 1.018]
weights = [1/len(alphas)]*len(alphas)
sub = 0.

te = create_dt(False)
te_feat = te[(te.date >= fday - timedelta(days=max_lags))].copy()
create_fea(te_feat)
for icount, (alpha, weight) in enumerate(zip(alphas, weights)):
    cols = [f"F{i}" for i in range(1,29)]
    for tdelta in range(0, 28):
        day = fday + timedelta(days=tdelta)
        print(tdelta, day)
        # tst = te[(te.date >= day - timedelta(days=max_lags)) & (te.date <= day)].copy()
        tst = te_feat[(te_feat.date >= day - timedelta(days=max_lags)) & (te_feat.date <= day)].copy()
        tst = tst.loc[tst.date == day , train_cols]
        te.loc[te.date == day, "sales"] = alpha*m_lgb.predict(tst) # magic multiplier by kyakovlev



    te_sub = te.loc[te.date >= fday, ["id", "sales"]].copy()
#     te_sub.loc[te.date >= fday+ timedelta(days=h), "id"] = te_sub.loc[te.date >= fday+timedelta(days=h), 
#                                                                           "id"].str.replace("validation$", "evaluation")
    te_sub["F"] = [f"F{rank}" for rank in te_sub.groupby("id")["id"].cumcount()+1]
    te_sub = te_sub.set_index(["id", "F" ]).unstack()["sales"][cols].reset_index()
    te_sub.fillna(0., inplace = True)
    te_sub.sort_values("id", inplace = True)
    te_sub.reset_index(drop=True, inplace = True)
    te_sub.to_csv(os.path.join(outdir, f"submission_{icount}.csv"),index=False)
    if icount == 0 :
        sub = te_sub
        sub[cols] *= weight
    else:
        sub[cols] += te_sub[cols]*weight
    print(icount, alpha, weight)


sub2 = sub.copy()
sub2["id"] = sub2["id"].str.replace("validation$", "evaluation")
sub = pd.concat([sub, sub2], axis=0, sort=False)
sub.to_csv(os.path.join(outdir,"submission.csv"),index=False)

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,2,1,1,2011,d_1,0,0,0,0,0,0,0
1,2011-01-30,11101,3,2,1,2011,d_2,0,0,0,0,0,0,0
2,2011-01-31,11101,1,3,1,2011,d_3,0,0,0,0,0,0,0
3,2011-02-01,11101,5,4,2,2011,d_4,0,0,0,0,1,1,0
4,2011-02-02,11101,6,5,2,2011,d_5,0,0,0,0,1,0,1


,id,item_id,dept_id,store_id,cat_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1856,4.0,2016-02-27,11605,2,1,2,2016,0,0,0,0,0,0,0,8.26
1,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1857,2.0,2016-02-28,11605,3,2,2,2016,0,0,0,0,0,0,0,8.26
2,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1858,0.0,2016-02-29,11605,1,3,2,2016,0,0,0,0,0,0,0,8.26
3,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1859,2.0,2016-03-01,11605,5,4,3,2016,0,0,0,0,1,1,0,8.26
4,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,d_1860,0.0,2016-03-02,11605,6,5,3,2016,0,0,0,0,1,0,1,8.26


0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2016-05-10 00:00:00
16 2016-05-11 00:00:00
17 2016-05-12 00:00:00
18 2016-05-13 00:00:00
19 2016-05-14 00:00:00
20 2016-05-15 00:00:00
21 2016-05-16 00:00:00
22 2016-05-17 00:00:00
23 2016-05-18 00:00:00
24 2016-05-19 00:00:00
25 2016-05-20 00:00:00
26 2016-05-21 00:00:00
27 2016-05-22 00:00:00
0 1.028 0.3333333333333333
0 2016-04-25 00:00:00
1 2016-04-26 00:00:00
2 2016-04-27 00:00:00
3 2016-04-28 00:00:00
4 2016-04-29 00:00:00
5 2016-04-30 00:00:00
6 2016-05-01 00:00:00
7 2016-05-02 00:00:00
8 2016-05-03 00:00:00
9 2016-05-04 00:00:00
10 2016-05-05 00:00:00
11 2016-05-06 00:00:00
12 2016-05-07 00:00:00
13 2016-05-08 00:00:00
14 2016-05-09 00:00:00
15 2

In [37]:
print(outdir)
print(model_path)
sub.head(10)

m5_competition/res/lgbm/LGBM-GPU/19-Jun-2020-19-l28-w28-2
m5_competition/res/lgbm/LGBM-GPU/19-Jun-2020-19-l28-w28


F,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,1.042104,0.943861,0.933698,0.919800,1.181432,1.375546,1.229174,0.083378,0.082513,0.097695,0.092828,0.110166,0.115435,0.096700,0.103317,0.099854,0.105770,0.109006,0.119443,0.131575,0.125876,0.115423,0.101788,0.101659,0.110065,0.121317,0.131381,0.130195
1,FOODS_1_001_CA_2_validation,1.239575,1.305299,1.180224,1.553394,1.484169,1.655140,1.662987,0.124991,0.121748,0.116933,0.127526,0.137774,0.161653,0.127928,0.112944,0.120286,0.126935,0.126718,0.131056,0.147068,0.149901,0.102025,0.105760,0.099415,0.097909,0.109102,0.121466,0.127089
2,FOODS_1_001_CA_3_validation,1.167082,1.105432,1.091877,1.078158,1.139219,1.265174,0.998170,0.149598,0.152703,0.159316,0.167452,0.181977,0.165035,0.133132,0.124830,0.123816,0.119215,0.120982,0.121479,0.158191,0.151969,0.122539,0.122891,0.122080,0.128217,0.125781,0.137805,0.202059
3,FOODS_1_001_CA_4_validation,0.425349,0.371461,0.358326,0.361823,0.433418,0.480138,0.530168,0.096082,0.096120,0.097214,0.104311,0.108590,0.100541,0.090159,0.092461,0.091261,0.095435,0.101519,0.109151,0.101251,0.102369,0.088994,0.083463,0.079443,0.084602,0.093794,0.084030,0.081573
4,FOODS_1_001_TX_1_validation,0.184943,0.181093,0.169461,0.171242,0.180233,0.188157,0.197998,0.107405,0.110533,0.107065,0.121306,0.131232,0.140824,0.119694,0.111233,0.108895,0.109652,0.116104,0.124446,0.127323,0.122037,0.090221,0.086333,0.088361,0.094488,0.102917,0.107668,0.109171
5,FOODS_1_001_TX_2_validation,0.483904,0.458230,0.458624,0.412521,0.401615,0.485523,0.475050,0.092413,0.095349,0.095726,0.127893,0.136660,0.138621,0.122502,0.109582,0.105806,0.108926,0.106536,0.113480,0.107755,0.112524,0.087423,0.087439,0.087048,0.098236,0.131467,0.128766,0.135484
6,FOODS_1_001_TX_3_validation,0.431344,0.391718,0.390350,0.461200,0.499533,0.524495,0.552812,0.093206,0.093818,0.090108,0.096505,0.110389,0.108546,0.092215,0.089441,0.089115,0.091638,0.095934,0.113649,0.110902,0.112768,0.089672,0.088362,0.087682,0.097434,0.101969,0.108136,0.113187
7,FOODS_1_001_WI_1_validation,0.309059,0.377389,0.346116,0.353661,0.411420,0.983092,0.634912,0.096587,0.101049,0.103803,0.117179,0.128152,0.135178,0.113250,0.100938,0.098482,0.099445,0.109363,0.124985,0.137408,0.124670,0.097633,0.096917,0.092070,0.099681,0.107465,0.122708,0.107375
8,FOODS_1_001_WI_2_validation,0.304670,0.348932,0.358362,0.340805,0.409735,0.459941,0.321170,0.099732,0.105793,0.095150,0.104422,0.111617,0.105668,0.111456,0.103736,0.087485,0.101106,0.100546,0.099908,0.108757,0.106713,0.092800,0.088189,0.102534,0.106532,0.114772,0.114590,0.109211
9,FOODS_1_001_WI_3_validation,0.230103,0.224199,0.206582,0.213842,0.246227,0.450138,0.282550,0.089717,0.093356,0.093272,0.108914,0.117441,0.120369,0.107249,0.092612,0.086114,0.095427,0.109220,0.115076,0.128655,0.122510,0.096042,0.093324,0.088677,0.093860,0.108627,0.114157,0.096540


In [38]:
sub.id.nunique(), sub["id"].str.contains("validation$").sum()

(60980, 30490)

In [39]:
sub.shape

(60980, 29)